In [ ]:
from dash import Dash, dcc, html
from dash.dependencies import Output, Input, State
from dash.exceptions import PreventUpdate
from dash_bootstrap_templates import load_figure_template
import dash_bootstrap_components as dbc

import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import math

dbc_css = 'https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css'

# Instantiate the app with an out-of-the-box theme
app = Dash(__name__, external_stylesheets=[dbc.themes.LUMEN, dbc_css])
load_figure_template('lumen')

# Read in entire natural disasters dataset
events = pd.read_csv('data/natural_disasters.csv')

# Store year values to help with plots + filters
years = list(set(events['Year']))
min_year = min(years)
max_year = max(years)

# Radio select options on sunburst chart
hierarchy = {
    'Geography': ['Continent', 'Region', 'Disaster Subgroup', 'Disaster Type'],
    'Event': ['Disaster Subgroup', 'Continent', 'Region', 'Disaster Type']
}

# Data table column names and sort orders
data_table_columns = dict(
    zip(
        ['Year', 'Country', 'Disaster Type', 'Event Name', "Total Damages ('000 US$)"],
        [['Year', False], ['Country', True], ['Event Type', True], ['Event Name', True], ['Total Damages (USD)', False]]
    )
)

#--------------------------------------------------#
#---------------- Dashboard layout ----------------#
#--------------------------------------------------#

app.layout = html.Div([

    html.H1(
        '🌋 Natural Disasters Dashboard ⛈',
        className='page-header'
    ),

    dbc.Container(
        className='filters',
        children=[
            dbc.Row([
                dbc.Col(
                    style={'margin-top': '10px'},
                    children=[
                        dbc.Button(
                            id='button-clear-filters',
                            color=None,
                            title='Reset all filters and selections',
                            className='dbc',
                            style={'padding': '0px', 'height': '43px'},
                            children=[
                                html.P(
                                    style={'fontSize': '26px'},
                                    children=['🔄']
                                )
                            ]
                        )
                    ]
                ),
                dbc.Col([
                    dcc.Checklist(
                        id='checklist-all-continents',
                        options=[{'label': 'Select All Continents', 'value': 'select_all'}],
                        value=[],
                        inline=True,
                        className='filter-text'
                    ),
                    dcc.Dropdown(
                        id='dropdown-continent',
                        placeholder='Select Continents...',
                        options=events['Continent'].sort_values().unique(),
                        multi=True,
                        className='dropdown'
                    )
                ]),
                dbc.Col([
                    dcc.Checklist(
                        id='checklist-all-regions',
                        options=[{'label': 'Select All Regions', 'value': 'select_all'}],
                        value=[],
                        inline=True,
                        className='filter-text'
                    ),
                    dcc.Dropdown(
                        id='dropdown-region',
                        placeholder='Select Regions...',
                        options=events['Region'].sort_values().unique(),
                        multi=True,
                        className='dropdown'
                    )
                ]),
                dbc.Col([
                    dcc.Checklist(
                        id='checklist-all-countries',
                        options=[{'label': 'Select All Countries', 'value': 'select_all'}],
                        value=[],
                        inline=True,
                        className='filter-text'
                    ),
                    dcc.Dropdown(
                        id='dropdown-country',
                        placeholder='Select Countries...',
                        options=events['Country'].sort_values().unique(),
                        multi=True,
                        value=[],
                        className='dropdown'
                    )
                ]),
                dbc.Col([
                    dcc.Checklist(
                        id='checklist-all-event-categories',
                        options=[{'label': 'Select All Event Categories', 'value': 'select_all'}],
                        value=[],
                        inline=True,
                        className='filter-text'
                    ),
                    dcc.Dropdown(
                        id='dropdown-event-category',
                        placeholder='Select Event Categories...',
                        options=events['Disaster Subgroup'].sort_values().unique(),
                        multi=True,
                        className='dropdown'
                    )
                ]),
                dbc.Col([
                    dcc.Checklist(
                        id='checklist-all-events',
                        options=[{'label': 'Select All Event Types', 'value': 'select_all'}],
                        value=[],
                        inline=True,
                        className='filter-text'
                    ),
                    dcc.Dropdown(
                        id='dropdown-event-type',
                        placeholder='Select Event Types...',
                        options=events['Disaster Type'].sort_values().unique(),
                        multi=True,
                        className='dropdown'
                    ) 
                ]),
            ]),
            dbc.Row([
                dbc.Col(
                    width=1,
                    children=[
                        dcc.Interval(
                            id='interval-year',
                            interval=1000, # milliseconds
                            disabled=True
                        ),
                        dbc.Button(
                            'Play',
                            id='button-play',
                            title='Autoplay through the years',
                            n_clicks=0,
                            color=None
                        )
                    ]
                ),
                dbc.Col(
                    width=11,
                    children=[
                        dcc.Slider(
                            id='slider-year',
                            min=events['Year'].min(),
                            max=events['Year'].max(),
                            step=1,
                            value=events['Year'].max(),
                            marks={i: f'{i}' for i in range(events['Year'].min(), events['Year'].max(), 5)},
                            className='dbc'
                        )
                    ]
                )
            ])
        ]
    ),

    dcc.Tabs(
        style={'fontSize': '16px'},
        children=[
            dcc.Tab(
                label='🌎 World Explorer',
                value='tab-1',
                children=[
                    html.Div(
                        children=[
                            html.H3(
                                id='title-map-line',
                                style=dict(
                                    textAlign='center',
                                    margin=dict(l=0, r=0, t=0, b=0)
                                )
                            ),
                            dbc.Row([
                                dbc.Col(
                                    width=6,
                                    children=[
                                        dcc.Graph(
                                            id='vis-map',
                                            hoverData={'points': [{'customdata': [None]}]} # Default to all countries upon load
                                        )
                                    ]
                                ),
                                dbc.Col(
                                    width=6,
                                    children=[dcc.Graph(id='vis-line')]
                                )
                            ])
                        ]
                    )
                ]
            ),
            dcc.Tab(
                label='⏬ Hierarchy Drilldown',
                value='tab-2',
                children=[
                    dbc.Row([
                        dbc.Col(
                            width=4,
                            style={'display': 'flex', 'justify-content': 'center', 'margin-top': '10px'},
                            children=[
                                html.P(
                                    style={'font-size': '14px'},
                                    children=['Order Hierarchy By: ']
                                ),
                                dbc.Card(
                                    style={'margin-left': '10px', 'width': '200px', 'height': '28px'},
                                    children=[
                                        dcc.RadioItems(
                                            id='radio-hierarchy-option',
                                            value='Geography',
                                            inline=True,
                                            style={'text-align': 'center', 'backgroundColor': '#f9f9f9'},
                                            options=[
                                                {
                                                    'label': html.Span('Geography', className='radio-button', style={'margin-left': '1px', 'margin-right': '10px'}),
                                                    'value': 'Geography'
                                                },
                                                {
                                                    'label': html.Span('Event', className='radio-button', style={'margin-left': '1px', 'margin-right': '0px'}),
                                                    'value': 'Event'
                                                }
                                            ],
                                        )
                                    ]
                                )
                            ]
                        ),
                        dbc.Col(
                            width=4,
                            style={'text-align': 'center'},
                            children=[
                                html.H3(id='title-sunburst-column')
                            ]
                        ),
                        dbc.Col(
                            width=4,
                            children=[]
                        )
                    ]),
                    dbc.Row([
                        dbc.Col(
                            width=4,
                            children=[
                                dcc.Store(id='store-vis-sunburst'),
                                dcc.Graph(id='vis-sunburst')
                            ]
                        ),
                        dbc.Col(
                            width=8,
                            children=[
                                dcc.Graph(id='vis-column')
                            ]
                        )
                    ])
                ]
            ),
            dcc.Tab(
                label='🔎 Event Details',
                value='tab-3',
                children=[
                    dbc.Row([
                        dbc.Col(
                            width=2,
                            children=[
                                dbc.Card(
                                    style={'margin-left': '10px', 'margin-top': '10px'},
                                    children=[
                                        dcc.RadioItems(
                                            id='radio-geography',
                                            value='Continent',
                                            inline=True,
                                            style={'text-align': 'center', 'backgroundColor': '#f9f9f9'},
                                            options=[
                                                {
                                                    'label': html.Span('Continent', className='radio-button', style={'margin-left': '1px', 'margin-right': '10px'}),
                                                    'value': 'Continent'
                                                },
                                                {
                                                    'label': html.Span('Region', className='radio-button', style={'margin-left': '1px', 'margin-right': '10px'}),
                                                    'value': 'Region'
                                                },
                                                {
                                                    'label': html.Span('Country', className='radio-button', style={'margin-left': '1px', 'margin-right': '0px'}),
                                                    'value': 'Country'
                                                }
                                            ]
                                        )
                                    ]
                                )
                            ]
                        ),
                        dbc.Col(
                            width=2,
                            children=[
                                dbc.Card(
                                    style={'margin-top': '10px'},
                                    children=[
                                        dcc.RadioItems(
                                            id='radio-disaster-category',
                                            value='Disaster Subgroup',
                                            inline=True,
                                            style={'text-align': 'center', 'backgroundColor': '#f9f9f9'},
                                            options=[
                                                {
                                                    'label': html.Span('Event Category', className='radio-button', style={'margin-left': '1px', 'margin-right': '10px'}),
                                                    'value': 'Disaster Subgroup'
                                                },
                                                {
                                                    'label': html.Span('Event Type', className='radio-button', style={'margin-left': '1px', 'margin-right': '0px'}),
                                                    'value': 'Disaster Type'
                                                }
                                            ]
                                        )
                                    ]
                                )
                            ]
                        ),
                        dbc.Col(
                            width=4,
                            style={'text-align': 'center'},
                            children=[
                                html.H3(id='title-bar-table')
                            ]
                        ),
                        dbc.Col(
                            width=4,
                            style={'display': 'flex', 'justify-content': 'right', 'margin-top': '10px'},
                            children=[
                                html.P(
                                    style={'font-size': '14px'},
                                    children=['Order Table By: ']
                                ),
                                dcc.Dropdown(
                                    id='dropdown-sort-order',
                                    placeholder='Select a column...',
                                    options=[{'label': v[0], 'value': k} for k, v in data_table_columns.items()],
                                    value='Year',
                                    multi=False,
                                    className='dropdown',
                                    style={'margin-left': '5px', 'margin-right': '25px', 'width': '175px', 'height': '28px'}
                                ),
                                dcc.Checklist(
                                    id='checklist-sort-order',
                                    options=[{'label': 'Ascending', 'value': True}],
                                    value=[],
                                    inline=True,
                                    className='filter-text',
                                    style={'margin-right': '25px'}
                                )
                            ]
                        )
                    ]),
                    dbc.Row([
                        dbc.Col(
                            width=7,
                            children=[
                                dcc.Graph(id='vis-bar'),
                                dcc.Store(id='store-vis-bar')
                            ]
                        ),
                        dbc.Col(
                            width=5,
                            children=[
                                html.P(
                                    id='title-data-table',
                                    style={'fontSize': '12px', 'fontFamily': 'arial', 'fontStyle': 'italic'}
                                ),
                                html.Div(
                                    id='data-table',
                                    style={
                                        'margin-right': '10px',
                                        'height': '375px',
                                        'overflow-y': 'scroll',
                                        'padding': '5px',
                                        'border': '1px solid #ccc',
                                        'backgroundColor': '#f9f9f9'
                                    }
                                )
                            ]
                        )
                    ])
                ]
            )
        ]
    ),

    html.Div(
        className='page-footer',
        children=[
            html.Div(
                className='page-footer-left',
                children=[
                    html.P(
                        className='page-footer-text-left',
                        children=[
                            'The data in this dashboard is taken from the ', html.B(html.I('EOSDIS Natural Disaster dataset')), ', available on ',
                            html.B(
                                html.A(
                                    children=['kaggle.com'],
                                    href='https://www.kaggle.com/datasets/brsdincer/all-natural-disasters-19002021-eosdis/data',
                                    target='blank',
                                )
                            ),
                            '.', html.Br(),
                            'Follow the adjacent GitHub link for the complete details about this project.'
                        ]
                    )
                ]
            ),
            html.Div(
                className='page-footer-center',
                children=[
                    html.A(
                        href='https://www.linkedin.com/in/tasmitaros/',
                        target='_blank',
                        rel='noopener noreferrer',
                        children=[
                            html.Img(
                                src='assets/linkedin_transparent.png',
                                className='page-footer-images'
                            )
                        ]
                    ),
                    html.A(
                        href='https://github.com/tasmitaros/natural_disasters',
                        target='blank',
                        rel='noopener noreferrer',
                        children=[
                            html.Img(
                                src='assets/github_transparent.png',
                                className='page-footer-images'
                            )
                        ]
                    )
                ]
            ),
            html.Div(
                className='page-footer-right',
                children=[
                    html.P(
                        className='page-footer-text-right',
                        children=[
                            'This dashboard was written in Python',
                            html.Br(),
                            'using the Plotly + Dash libraries'
                        ]
                    )
                ]
            )
        ]
    )

])

#--------------------------------------------------#
#---------------- Helper functions ----------------#
#--------------------------------------------------#

def is_empty(value):
    """
    Checks if the value is set or not.
    Used for determining if values have been selected in the dropdown box filters.
    """
    if value is None:
        return True
    
    elif type(value) != int and len(value) == 0:
        return True
    
    if value == []:
        return True
    
    else:
        return False

def apply_df_filters(df, continent=None, region=None, country=None, disaster_subgroup=None, disaster_type=None, year=None):
    """
    Conditionally applies filters from the dropdown boxes to the target dataframe.
    """
    df = events
    
    # Conditionally apply filters if values are selected
    df = df.query(f'Continent == {"Continent" if is_empty(continent) else "@continent"}')
    df = df.query(f'Region == {"Region" if is_empty(region) else "@region"}')
    df = df.query(f'Country == {"Country" if is_empty(country) else "@country"}')
    df = df.query(f'`Disaster Subgroup` == {"`Disaster Subgroup`" if is_empty(disaster_subgroup) else "@disaster_subgroup"}')
    df = df.query(f'`Disaster Type` == {"`Disaster Type`" if is_empty(disaster_type) else "@disaster_type"}')
    df = df.query(f'Year <= {max_year if is_empty(year) else "@year"}')

    return df

def get_line_series_categories(df, continent=None, region=None, country=None, disaster_subgroup=None, disaster_type=None):
    """
    Create a unique concatenated string value for the combinations of 
    selected filters in the dataframe, each representing one series on 
    the line chart.
    """
    group_by_cols = []

    # Dynamically add the filters to group the dataframe by, based on the selected filters
    if not is_empty(disaster_type):
        group_by_cols.insert(0, 'Disaster Type')
    elif not is_empty(disaster_subgroup):
        group_by_cols.insert(0, 'Disaster Subgroup')
        
    if not is_empty(country):
        group_by_cols.insert(0, 'Country')
    elif not is_empty(region):
        group_by_cols.insert(0, 'Region')
    elif not is_empty(continent):
        group_by_cols.insert(0, 'Continent')

    if is_empty(disaster_type) and is_empty(disaster_subgroup) and is_empty(country) and is_empty(region) and is_empty(continent): # Apply default label when not filters applied
        df['Category'] = 'All'
    else: # Concat all group by cols into a single col because the 'color' argument only accepts 1 col
        df['Category'] = df[group_by_cols].astype(str).agg('-'.join, axis=1)

    # Apply cumulative sum over the group by cols (Category) and order by Year ascending
    df = df.groupby(['Year', 'Category'], as_index=False).agg(**{'No. Events': ('Year', 'count')}).sort_values(['Category', 'Year'])
    df['Total Events'] = df.groupby('Category', as_index=False)['No. Events'].cumsum()
    df = df.sort_values(['Total Events', 'Category', 'Year'], ascending=[False, True, True])

    return df

def get_colour_palette(colour_palette, num_categories):
    """
    Compiles a colour palette for the number of series present in the visual.
    Where the number of series exceeds the number of colours in the colour palette, the colours will be cycled through from the start.
    """
    all_colours = []

    total_colours = len(colour_palette) # Total number of colours in the selected colour palette
    
    for i in range(0, num_categories):
        offset = math.floor(i / total_colours) * total_colours # How many times the colour palette has been cycled through
        index = i - offset # Always return a number between min and max of the colour palette's length
        all_colours.append(colour_palette[index])

    return all_colours

def get_plural_name(name):
    """
    Returns the plural version of the specified string
    """
    name = name.lower()
    final_letter = name[len(name)-1]

    if final_letter == 'y':
        name = name[:-1] + 'ies'
    else:
        name += 's'

    return name

def get_empty_table():
    """
    Returns an empty data table with a specified schema to prevent 
    empty space on the dashboard where there is no hover data
    """
    columns = [row[0] for row in list(data_table_columns.values())]
    empty_df = pd.DataFrame({col: ['​'] * 9 for col in columns}) # Use zero-width character to ensure row height is the same as when the table contains data
    tbl = dbc.Table.from_dataframe(
        empty_df,
        striped=True,
        bordered=True,
        hover=True,
        color='Dark',
        className='data-table',
        style={'textOverflow': 'ellipsis'}
    )
    return tbl

#--------------------------------------------------#
#----------------- Filter buttons -----------------#
#--------------------------------------------------#

@app.callback(
    Output('dropdown-continent', 'value', allow_duplicate=True),
    Output('dropdown-region', 'value', allow_duplicate=True),
    Output('dropdown-country', 'value', allow_duplicate=True),
    Output('dropdown-event-category', 'value', allow_duplicate=True),
    Output('dropdown-event-type', 'value', allow_duplicate=True),
    Output('checklist-all-continents', 'value'),
    Output('checklist-all-regions', 'value'),
    Output('checklist-all-countries', 'value'),
    Output('checklist-all-event-categories', 'value'),
    Output('checklist-all-events', 'value'),
    Output('radio-hierarchy-option', 'value'),
    Output('radio-geography', 'value'),
    Output('radio-disaster-category', 'value'),
    Output('dropdown-sort-order', 'value'),
    Output('checklist-sort-order', 'value'),
    Output('vis-bar', 'hoverData'),
    Output('slider-year', 'value', allow_duplicate=True),
    Input('button-clear-filters', 'n_clicks'),
    prevent_initial_call=True
)
def clear_filters_button(n_clicks):
    """
    Clears all selected filters and resets all selected view options
    on all pages in the dashboard.
    """
    return [], [], [], [], [], [], [], [], [], [], 'Geography', 'Continent', 'Disaster Subgroup', 'Year', [], None, max_year

@app.callback(
    Output('button-play', 'children'),
    Output('button-play', 'className'),
    Output('interval-year', 'disabled'),
    Output('button-play', 'n_clicks'),
    Input('button-play', 'n_clicks'),
    State('interval-year', 'disabled'),
    Input('slider-year', 'value')
)
def play_button(n_clicks, disabled, year):
    """
    Toggle between 'Play' and 'Stop' on the year slider when 
    the button is clicked
    """
    if n_clicks % 2 == 0: # Stop on even number of clicks
        button_text = 'Play'
        disabled = True
        button_style = 'button-play'
    elif year == max_year and disabled == False: # Stop when the slider reaches the final year
        n_clicks += 1 # Artifically click the button to maintain the odd/even sequence
        button_text = 'Play'
        disabled = True
        button_style = 'button-play'
    else: # Play on odd number of clicks
        button_text = 'Stop'
        disabled = False
        button_style = 'button-stop'

    return button_text, button_style, disabled, n_clicks

@app.callback(
    Output('slider-year', 'value'),
    Input('interval-year', 'n_intervals'),
    State('slider-year', 'value'),
    prevent_initial_call=True
)
def update_year_interval(n, selected_year):
    """
    Determines the logic for the button when in the 'Play' state. 
    Follows the specified year increment and ensures the final year 
    isn't missed.
    """
    step = 5 # Increment by this many years each time the interval is triggered

    if selected_year == max(years): # Go back to first year after final year
        index = years.index(min(years))
    elif selected_year + step > max(years): # Don't skip the final year
        index = years.index(max(years))
    else:
        index = (years.index(selected_year) + step) # Add the specified step

    index = index % len(years)
    year = years[index]

    return year

#--------------------------------------------------#
#------------- Conditional dropdowns --------------#
#--------------------------------------------------#

@app.callback(
    Output('dropdown-region', 'options'),
    Input('dropdown-continent', 'value')
)
def get_regions(continent):
    """
    Returns all the regions for the selected continent as options 
    in the region dropdown box.
    """
    if is_empty(continent):
        df = events
    else:
        df = events.query('Continent == @continent')

    return df['Region'].sort_values().unique()

@app.callback(
    Output('dropdown-country', 'options'),
    Input('dropdown-continent', 'value'),
    Input('dropdown-region', 'value')
)
def get_countries(continent, region):
    """
    Returns all the countries for the selected continent / region as options 
    in the country dropdown box.
    """
    df = events

    if is_empty(region):
        if is_empty(continent):
            df = events
        else:
            df = df.query('Continent == @continent')
    else:
        df = df.query('Region == @region')

    return df['Country'].sort_values().unique()

@app.callback(
    Output('dropdown-event-type', 'options'),
    Input('dropdown-event-category', 'value')
)
def get_event_types(disaster_subgroup):
    """
    Returns all the event types for the selected event categories as options 
    in the event types dropdown box.
    """
    df = events

    if is_empty(disaster_subgroup):
        df = events
    else:
        df = df.query('`Disaster Subgroup` == @disaster_subgroup')
    
    return df['Disaster Type'].sort_values().unique()

#--------------------------------------------------#
#--------------- Select All buttons ---------------#
#--------------------------------------------------#

@app.callback(
    Output('dropdown-continent', 'value', allow_duplicate=True),
    Input('checklist-all-continents', 'value'),
    prevent_initial_call=True
)
def select_all_continents(checkbox):
    """
    Adds / removes all the available continents in the 
    continents dropdown box when the checkbox is clicked.
    """
    if 'select_all' in checkbox:
        return events['Continent'].unique()
    else:
        []

@app.callback(
    Output('dropdown-region', 'value', allow_duplicate=True),
    Input('checklist-all-regions', 'value'),
    prevent_initial_call=True
)
def select_all_regions(checkbox):
    """
    Adds / removes all the available regions in the 
    regions dropdown box when the checkbox is clicked.
    """
    if 'select_all' in checkbox:
        return events['Region'].unique()
    else:
        return []

@app.callback(
    Output('dropdown-country', 'value', allow_duplicate=True),
    Input('checklist-all-countries', 'value'),
    prevent_initial_call=True
)
def select_all_countries(checkbox):
    """
    Adds / removes all the available countries in the 
    countries dropdown box when the checkbox is clicked.
    """
    if 'select_all' in checkbox:
        return events['Country'].unique()
    else:
        return []

@app.callback(
    Output('dropdown-event-category', 'value', allow_duplicate=True),
    Input('checklist-all-event-categories', 'value'),
    prevent_initial_call=True
)
def select_all_event_categories(checkbox):
    """
    Adds / removes all the available event categories in the 
    event categories dropdown box when the checkbox is clicked.
    """
    if 'select_all' in checkbox:
        return events['Disaster Subgroup'].unique()
    else:
        return []

@app.callback(
    Output('dropdown-event-type', 'value', allow_duplicate=True),
    Input('checklist-all-events', 'value'),
    prevent_initial_call=True
)
def select_all_events(checkbox):
    """
    Adds / removes all the available event types in the 
    event types dropdown box when the checkbox is clicked.
    """
    if 'select_all' in checkbox:
        return events['Disaster Type'].unique()
    else:
        return []

#--------------------------------------------------#
#-------------------- Visuals ---------------------#
#--------------------------------------------------#

@app.callback(
    Output('vis-map', 'figure'),
    Output('title-map-line', 'children'),
    Input('dropdown-continent', 'value'),
    Input('dropdown-region', 'value'),
    Input('dropdown-country', 'value'),
    Input('dropdown-event-category', 'value'),
    Input('dropdown-event-type', 'value'),
    Input('slider-year', 'value')
)
def plot_map(continent, region, country, disaster_subgroup, disaster_type, year):

    df = apply_df_filters(events, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type, year=year)
    df = df.groupby(['Continent', 'Region', 'Country'], as_index=False).agg(**{'No. Events': ('Country', 'count')})

    fig = px.choropleth(
        df,
        locations='Country',
        locationmode='country names',
        color='No. Events',
        color_continuous_scale='reds'
    )

    fig.data[0].customdata = df[['Continent', 'Region', 'No. Events']].to_numpy()

    fig.data[0].hovertemplate = (
        'Continent=%{customdata[0]}<br>'
        'Region=%{customdata[1]}<br>'
        'Country=<span style="color:#FFC000;font-weight:bold">%{location}</span><br>'
        'No. Events=%{z:,}'
        '<extra></extra>'
    )

    title = '<span style="font-size:20px">No. Events By Country'
    subtitle = '<span style="font-size:12px;font-style:italic">Scroll up and down to zoom in and out of any region of the map'

    fig.update_layout(
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        uirevision=True, # Prevent reset zoom on callback
        margin=dict(l=10, r=0, t=50, b=10),
        autosize=True,
        coloraxis_colorbar=dict(
            thickness=15,
            tickformat=',',
            titlefont=dict(weight='bold'),
            tickfont=dict(weight='bold')
        ),
        title=f'{title}<br>{subtitle}'
    )

    title = f'1900 - {year}'

    return fig, title

@app.callback(
    Output('vis-line', 'figure'),
    Input('dropdown-continent', 'value'),
    Input('dropdown-region', 'value'),
    Input('dropdown-country', 'value'),
    Input('dropdown-event-category', 'value'),
    Input('dropdown-event-type', 'value'),
    Input('slider-year', 'value'),
    Input('button-play', 'n_clicks')
)
def plot_line(continent, region, country, disaster_subgroup, disaster_type, year, n_clicks):

    # Apply all selected filters on the page
    df = apply_df_filters(events, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type, year=year)

    # Exit function if no data for applied filters
    if df.empty:
        raise PreventUpdate

    df = get_line_series_categories(df, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type)

    colour_palette = px.colors.qualitative.Plotly

    fig = px.line(
        df,
        x='Year',
        y='Total Events',
        markers=True,
        color='Category',
        color_discrete_sequence=colour_palette,
    )

    last_points = df.sort_values(['Year', 'Total Events'], ascending=[True, False]).groupby('Category').tail(1)
    all_categories = df['Category'].unique()

    # Map each line series to its colour to conditionally colour the series labels
    color_map = dict(zip(
        all_categories,
        get_colour_palette(colour_palette, len(all_categories))
    ))

    annotations = [
        dict(
            x=row['Year'],
            y=row['Total Events'],
            text=row['Category'],
            showarrow=False,
            align='left',
            xanchor='left',
            font=dict(
                family='arial',
                size=12,
                color=color_map[row['Category']]
            )
        )
        for _, row in last_points.iterrows()
    ]

    fig.update_traces(hovertemplate=None)

    title = '<span style="font-size:20px">No. Events By Year'
    subtitle = '<span style="font-size:12px;font-style:italic">Apply filters above to expand the number of series shown'

    fig.update_layout(
        paper_bgcolor='rgba(0, 0, 0, 0)',
        plot_bgcolor='rgba(0, 0, 0, 0)',
        annotations=annotations,
        yaxis_tickformat=',',
        showlegend=False,
        autosize=True,
        hovermode='x',
        hoverlabel=dict(
            font=dict(color='white')
        ),
        margin=dict(l=10, r=10, t=40, b=60),
        title=f'{title}<br>{subtitle}'
    )

    fig.update_xaxes(
        range=[
            min_year,
            max_year + 20 # Allow some space for the series labels
        ]
    )

    # Only apply custom y-axis scale when play button is active
    if n_clicks % 2 != 0:
        df_all_years = apply_df_filters(events, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type)
        df_all_years = get_line_series_categories(df_all_years, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type)
        max_y = max(df_all_years['Total Events'])
        fig.update_yaxes(
            range=[
                round(0 - (max_y * 0.075), 0),
                round(max_y * 1.05, 0)
            ]
        )

    return fig

@app.callback(
    Output('vis-sunburst', 'figure'),
    Output('title-sunburst-column', 'children'),
    Input('dropdown-continent', 'value'),
    Input('dropdown-region', 'value'),
    Input('dropdown-country', 'value'),
    Input('dropdown-event-category', 'value'),
    Input('dropdown-event-type', 'value'),
    Input('slider-year', 'value'),
    Input('radio-hierarchy-option', 'value')
)
def plot_sunburst(continent, region, country, disaster_subgroup, disaster_type, year, hierarchy_option):

    df = apply_df_filters(events, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type, year=year)

    if df.empty:
        raise PreventUpdate

    columns = hierarchy.get(hierarchy_option)

    df = df.groupby(columns, as_index=False).agg(**{'No. Events': ('Country', 'count')})

    fig = px.sunburst(
        df,
        path=columns,
        values='No. Events'
    )

    page_title = f'1900 - {year}'
    title = '<span style="font-size:20px">Proportion of Events'
    subtitle = '<span style="font-size:12px;font-style:italic">Click on any slice in the chart to drill-down, or on the centre slice to drill-up'

    fig.update_layout(
        title=dict(
            text=f'{title}<br>{subtitle}',
            x=0.5
        )
    )

    for trace in fig.data:
        trace.hovertemplate = (
            'Category=%{label}'
            '<br>'
            'Parent=%{parent}'
            '<br>'
            'No. Events=%{value:,}'
            '<extra></extra>'
        )

    return fig, page_title

@app.callback(
    Output('vis-column', 'figure'),
    Input('dropdown-continent', 'value'),
    Input('dropdown-region', 'value'),
    Input('dropdown-country', 'value'),
    Input('dropdown-event-category', 'value'),
    Input('dropdown-event-type', 'value'),
    Input('slider-year', 'value'),
    Input('radio-hierarchy-option', 'value'),
    Input('vis-sunburst', 'clickData')
)
def plot_column(continent, region, country, disaster_subgroup, disaster_type, year, hierarchy_option, click_data):

    df = apply_df_filters(events, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type, year=year)

    if df.empty:
        raise PreventUpdate

    if not click_data:
        df = df

    else:
        
        path = click_data['points'][0].get('id').split('/') # Return array of selected path in sunburst chart
        currentCategory = click_data['points'][0].get('label') # The category that the user clicks
        previousCategory = click_data['points'][0].get('entry') # The previous category that the user clicked

        if currentCategory == previousCategory: # These 2 variables are equal when the user drills up, so we want to drop the last category in the path
            path = path[:-1]
        
        columns = hierarchy.get(hierarchy_option)

        # Dynamically build query
        query = ''
        if len(path) == 0:
            query = 'Continent == Continent' # Return everything
        if len(path) >= 1:
            query += f' and `{columns[0]}` in @path'
        if len(path) >= 2:
            query += f' and `{columns[1]}` in @path'
        if len(path) >= 3:
            query += f' and `{columns[2]}` in @path'
        if len(path) >= 4:
            query += f' and `{columns[3]}` in @path'
        
        query = query.lstrip(' and ') # Remove leading ' and ' string

        df = df.query(query)

    df = df.groupby(['Country', 'Disaster Type'], as_index=False).agg(**{'No. Events': ('Country', 'count')})
    df['Total Events'] = df.groupby('Country', as_index=False)['No. Events'].transform('sum')
    df = df.sort_values(['Total Events', 'Country', 'No. Events'], ascending=[False, True, False])
    
    fig = px.bar(
        df,
        x='Country',
        y='No. Events',
        color='Disaster Type',
        color_discrete_sequence=px.colors.qualitative.Plotly
    )

    # Ensure correct sort order of countries by total events descending
    fig.update_xaxes(
        categoryorder='array',
        categoryarray=df['Country'].unique()
    )

    title = f'<span style="font-size:20px">No. Events By Country'
    subtitle = '<span style="font-size:12px;font-style:italic">Click on any slice in the adjacent chart to apply filters'

    # Update tooltip
    for trace in fig.data:
        trace.customdata = df[df['Disaster Type'] == trace.name][['Disaster Type']].to_numpy()
        trace.hovertemplate = (
            '%{customdata[0]}=%{y}'
            '<extra></extra>'
        )

    fig.update_layout(
        title=f'{title}<br>{subtitle}',
        hovermode='x unified',
        legend=dict(title='Event Type')
    )

    fig.update_yaxes(tickformat=',')

    return fig

@app.callback(
    Output('vis-bar', 'figure'),
    Output('title-bar-table', 'children'),
    Output('store-vis-bar', 'data'),
    Input('dropdown-continent', 'value'),
    Input('dropdown-region', 'value'),
    Input('dropdown-country', 'value'),
    Input('dropdown-event-category', 'value'),
    Input('dropdown-event-type', 'value'),
    Input('slider-year', 'value'),
    Input('radio-geography', 'value'),
    Input('radio-disaster-category', 'value')
)
def plot_bar(continent, region, country, disaster_subgroup, disaster_type, year, geography, event_type):

    df = apply_df_filters(events, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type, year=year)
    df = df.groupby([geography, event_type], as_index=False).agg(**{'No. Events': (event_type, 'count')})

    # Exit function if no data for applied filters
    if df.empty:
        raise PreventUpdate

    all_geo = df[geography].unique()
    all_events = df[event_type].unique()
    all_combos = pd.MultiIndex.from_product([all_geo, all_events], names=[geography, event_type])
    df = df.set_index([geography, event_type]).reindex(all_combos, fill_value=0).reset_index()

    limit = 15

    df['Total Events'] = df.groupby(geography, as_index=False)['No. Events'].transform('sum')
    df['rank'] = df['Total Events'].rank(method='dense', ascending=False).astype(int)
    df = df.query('rank <= @limit')
    df = df.sort_values(geography, ascending=True)

    fig = px.histogram(
        df,
        x='No. Events',
        y=geography,
        color=event_type,
        color_discrete_sequence=px.colors.qualitative.Plotly,
        barnorm='percent'
    )

    page_title = f'1900 - {year}'
    title = f'<span style="font-size:20px">Proportion of Events By {geography}'
    subtitle = f'<span style="font-size:14px;font-style:italic">(showing top {limit} {get_plural_name(geography)} due to large volume)</span>' if max(df['rank']) >= limit else ''

    fig.update_layout(
        title=dict(
            text=f'{title}<br>{subtitle}',
            x=0.5
        ),
        legend=dict(
            title=None,
            orientation='h',
            yanchor='bottom',
            xanchor='center',
            y=-0.3,
            x=0.5
        ),
        hoverlabel=dict(
            font=dict(color='white')
        )
    )

    fig.update_xaxes(
        title='No. Events',
        showticklabels=False
    )

    # Add data labels to each series in the chart
    for trace in fig.data:

        filtered_df = df[df[event_type] == trace.name]

        trace.customdata = filtered_df['No. Events']
        trace.texttemplate = '%{customdata:,}'
        trace.hovertemplate = (
            'Geography=%{y}<br>'
            f'Event Type={trace.name}<br>'
            'No. Events=%{customdata:,}<br>'
            '% of Total=%{x:.0f}%<br>'
            '<extra></extra>'
        )
        trace.textposition = 'inside'
        trace.insidetextanchor = 'middle'
        trace.textangle = 0
        trace.textfont = dict(color='white')
        
    return fig, page_title, fig.to_plotly_json()

@app.callback(
    Output('data-table', 'children'),
    Output('title-data-table', 'children'),
    Input('dropdown-continent', 'value'),
    Input('dropdown-region', 'value'),
    Input('dropdown-country', 'value'),
    Input('dropdown-event-category', 'value'),
    Input('dropdown-event-type', 'value'),
    Input('slider-year', 'value'),
    Input('radio-geography', 'value'),
    Input('radio-disaster-category', 'value'),
    Input('dropdown-sort-order', 'value'),
    Input('checklist-sort-order', 'value'),
    Input('vis-bar', 'hoverData'),
    Input('store-vis-bar', 'data')
)
def plot_table(continent, region, country, disaster_subgroup, disaster_type, year, geography, event_type, sort_column, is_ascending, hover_data, bar_chart_data):

    title = ['Mouseover any bar in the adjacent chart to see the individual events', html.Br(), '​'] # Use zero-width character to force empty line break

    if bar_chart_data:
        bar_chart_fig = go.Figure(bar_chart_data)
    else:
        return get_empty_table(), title

    if hover_data:
        
        try:
            y = hover_data['points'][0]['y'] # Get the continent / region / country name
            curve_number = hover_data['points'][0]['curveNumber'] # Get the position of the disaster subgroup / disaster type
            category_name = bar_chart_fig.data[curve_number].name # Get the name of the disaster subgroup / disaster type
        except IndexError:
            raise PreventUpdate

        df = apply_df_filters(events, continent=continent, region=region, country=country, disaster_subgroup=disaster_subgroup, disaster_type=disaster_type, year=year)

        if df.empty:
            raise PreventUpdate

        df = df[df[event_type] == category_name]
        df = df[df[geography] == y]

        # Default sort order if dropdown is empty
        sort_columns = ['Year', "Total Damages ('000 US$)", 'Disaster Type', 'Country', 'Event Name']

        if sort_column:
            # Move the selected column to the front of the array
            sort_columns.remove(sort_column)
            sort_columns.insert(0, sort_column)

        # Get the corresponding True / False for each column
        sort_orders = [data_table_columns.get(col)[1] for col in sort_columns]

        if sort_column:
            # Update the selected asc / desc sort order in the array
            index = sort_columns.index(sort_column)
            sort_orders[index] = is_ascending[0] if not is_empty(is_ascending) else False

        # Apply the custom sort order
        df = df.sort_values(sort_columns, ascending=sort_orders)

        # Calculate the actual dollar amount and format with comma separators
        df["Total Damages ('000 US$)"] = df["Total Damages ('000 US$)"].apply(lambda x: f'${int(x)*1000:,}' if not pd.isna(x) else 'N/A')

        # Ensure the column labels are used instead of the values
        df.rename(columns={col:data_table_columns.get(col)[0] for col in sort_columns}, inplace=True)

        # Select only the required columns
        df = df[[row[0] for row in list(data_table_columns.values())]]

        num_records = df.shape[0]
        limit = 100

        if num_records == 0:
            tbl = get_empty_table()
        else:

            if num_records > limit:
                title.append(f'(Showing top {limit} due to large volume)')
                df = df.head(limit)

            tbl = dbc.Table.from_dataframe(
                df,
                striped=True,
                bordered=True,
                hover=True,
                color='Dark',
                className='data-table'
            )
        
        return tbl, title
    
    else:
        return get_empty_table(), title

if __name__ == '__main__':
    app.run_server(debug=True, port=8050)